# 사용예시

In [1]:
# 주피터 노트북 전체화면 설정
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}</style>"))

# 필요한 라이브러리 불러오기
from SynergyUtils import data2seq, optimize # 프로젝트 정의 함수 불러오기
from tensorflow import keras
import numpy as np
import pandas as pd

## 1. 모델 및 데이터 불러오기
- 모델은 케라스 라이브러리에서 제공하는 함수를 통해 불러옴  

In [2]:
# 저장된 모델 불러오기
model = keras.models.load_model('SRNN_model') # Stacked RNN (length = 4)

In [3]:
# 데이터 불러오기
data = pd.read_csv('data.csv')

data.tail()

,time,volt,cap
451318,2021-01-18 10:00:00,348.2,507.0
451319,2021-01-18 10:02:00,348.0,506.9
451320,2021-01-18 10:03:00,348.0,506.9
451321,2021-01-18 10:04:00,348.0,506.6
451322,2021-01-18 10:06:00,348.0,506.6


## 2. 데이터 전처리
- 정의된 전처리 함수를 통해 시퀀스 데이터 생성
- 전처리 함수의 작동 방식은 데이터프레임이 들어왔을 때 지정된 length만큼(디폴트 = 4)의 시퀀스 데이터 생성  

In [4]:
sample = data2seq(data, length = 4)
sample

array([[[0.26829268, 0.35368664],
        [0.26829268, 0.35357143],
        [0.26829268, 0.35357143],
        [0.26829268, 0.35322581]]])

In [6]:
sample.shape

(1, 4, 2)

## 3. 투입용량 예측
- 불러온 모델과 전처리된 데이터를 바탕으로 투입용량 예측  
- 모델의 계산 결과는 2차원의 배열이므로 적절한 인덱싱을 통해 값만 추출 

In [7]:
pred = model.predict(sample)[0,0]

pred

506.62207

## 4. 최적화 및 조정방안 도출 
- 정의된 최적화 함수를 통해 최종 조정방안 도출  
- (1) '현재 6개 리액터 작동 현황', (2) '각 리액터별 총 작동 횟수'가 사전에 정의되어 있어야 함

In [8]:
params_now = [1, 0, 0, 0, 1, 1]
n_used = [200, 150, 100, 50, 50, 100]

# 최종 아웃풋
# 왼쪽부터 일반 리액터 5개 최적 조정 방안 (1 : 투입, 0 : 개방)
# 가변형 리액터 탭 위치 (0 ~ 18, 0 : 개방)
# 최종 투입용량
optimize(pred, params_now, n_used)

(1.0, 0.0, 0.0, 0.0, 1.0, 4.0, 509.0)